In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [10]:
# Sample dataset
text = "Hello world. Welcome to the world of deep learning. Let's generate some text."
vocab = sorted(set(text))
char_to_index = {c: i for i, c in enumerate(vocab)}
index_to_char = {i: c for i, c in enumerate(vocab)}

# Parameters
seq_length = 10  # Length of input sequences
step = 1  # Step size for moving through the text

# Prepare input-output pairs
sentences = []
next_chars = []
for i in range(0, len(text) - seq_length, step):
    sentences.append(text[i:i + seq_length])
    next_chars.append(text[i + seq_length])

print(f"Number of sequences: {len(sentences)}")

Number of sequences: 67


In [12]:
# Vectorization
X = np.zeros((len(sentences), seq_length, len(vocab)), dtype=bool)
y = np.zeros((len(sentences), len(vocab)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1

In [14]:
# Model architecture
model = keras.Sequential([
    layers.LSTM(128, input_shape=(seq_length, len(vocab)), return_sequences=True),
    layers.LSTM(128),
    layers.Dense(len(vocab), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model
model.fit(X, y, batch_size=32, epochs=50)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 3.1735
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.1371
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.0973
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.0460
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.9730
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 2.8752
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 2.7731
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 2.7911
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.7962
Epoch 10/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.7203
Epoch 11/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.7312
Epoch 12/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.7176
Epoch 13/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.7107
Epoch 14/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.7214
Epoch 15/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.5957
Epoch 16/50
3/3 ━━━━━━━━━━━━━━━━━━

In [15]:
def generate_text(model, start_string, num_generate=100):
    input_eval = [char_to_index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    generated_text = []

    # Temperature for randomness in predictions
    temperature = 1.0

    model.reset_states()
    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)
        generated_text.append(index_to_char[predicted_id])

    return start_string + ''.join(generated_text)

In [16]:
# Generate text
start_string = "Hello "
generated_text = generate_text(model, start_string, num_generate=50)
print(generated_text)

AttributeError: 'Sequential' object has no attribute 'reset_states'